# Visualization notebook

This notebook is meant for visualizing stuff and testing code. 


## Imports and functions

In [1]:
import utils
import plotly.express as px
import numpy as np
import pandas as pd
import warnings
import spacy
import json
warnings.filterwarnings("ignore", category=FutureWarning)


# MUD: 1,071,477 authors

### Plotting by bin

In [ ]:
bin_nums = "1 2 3 4 5 6 7 8".split()

dev_bin_avg_docs = []




# manually recorded
HALF_FEATS_ACCS = [0.0, 0.02857142857142857, 0.08571428571428572, 0.11428571428571428, 0.22857142857142856, 0.22857142857142856, 0.22857142857142856, 0.4]
ALL_FEATS_ACCS = [0.0, 0.05714285714285714, 0.11428571428571428, 0.17142857142857143, 0.2, 0.2, 0.2857142857142857, 0.5428571428571428]


 

df = pd.DataFrame.from_dict({"half_features": HALF_FEATS_ACCS,
                             "all_features" : ALL_FEATS_ACCS,
                             "bin":bin_nums})


fig = px.line(df, x="bin", y=df.columns[0:2], template="plotly_dark", markers=True,)
fig.update_layout(title_text="Accuracy per bin", title_x=0.5)



# Summary stats

- Mean/std \# of tokens per document
- Mean/std \# of tokens per author
- Mean/std \# of documents per author

In [3]:
data = utils.load_json("data/pan/preprocessed/fixed_sorted_author.json")

def get_document_token_counts(data:dict[str, list]):
    token_counts = []
    for author_id in data.keys():
        for doc in data[author_id]:
            doc_tokens = doc.split()
            token_counts.append(len(doc_tokens))
    return token_counts

doc_token_counts = get_document_token_counts(data) 
print("Mean/std tokens per document")      
print(np.mean(doc_token_counts))
print(np.std(doc_token_counts))


def get_author_token_counts(data:dict[str, list[str]]) -> list[int]:
    
    author_to_token_counts = {}
    for author_id in data.keys():
        token_counts = []
        for doc in data[author_id]:
            doc_tokens = doc.split()
            token_counts.append(len(doc_tokens))
            
        author_to_token_counts[author_id] = sum(token_counts)
            
    return list(author_to_token_counts.values())


author_token_counts = get_author_token_counts(data)

print("\nMean/std tokens per author")
print(np.mean(author_token_counts))
print(np.std(author_token_counts))


def get_num_docs_per_author(data:dict[str, list[str]]) -> list[int]:
    author_to_doc_freq = {}
    for author_id in data.keys():
        author_to_doc_freq[author_id] = len(data[author_id])
    return list(author_to_doc_freq.values())

docs_per_author = get_num_docs_per_author(data)

print("\nMean/std document frequency per author")
print(np.mean(docs_per_author))
print(np.std(docs_per_author))


Mean/std tokens per document
247.99808978032473
220.32835614956986

Mean/std tokens per author
4636.678571428572
1772.7924311172276

Mean/std document frequency per author
18.696428571428573
6.114735255503541


# The following cells are for generating my potential CSE 564 dataset

In [4]:
from gram2vec.featurizers import GrammarVectorizer


def vectorize_all_data(data:dict, g2v:GrammarVectorizer) -> np.ndarray:
    """Vectorizes a dict of documents. Returns a matrix from all documents"""
    vectors = []
    for author_id in data.keys():
        for text in data[author_id]:
            grammar_vector = g2v.vectorize(text, return_vector=False)
            vectors.append(grammar_vector)
    try:
        return np.stack(vectors)
    except:
        return vectors

def get_authors(data:dict) -> list[int]:
    """Get all instances of authors from data"""
    authors = []
    for author_id in data.keys():
        for _ in data[author_id]:
            authors.append(author_id)
    return authors


data = utils.load_json("data/pan/preprocessed/fixed_sorted_author.json")
g2v = GrammarVectorizer()
authors = get_authors(data)
feature_vectors = vectorize_all_data(data, g2v)

GrammarVectorizer: Old logs detected. Clearing...
GrammarVectorizer: Done


In [16]:
import csv 
from collections import defaultdict

def get_vocab(path) -> list[str]:
    with open(path, "r") as fin:
        return fin.read().strip().split("\n")

pos_unigrams  = get_vocab("vocab/static/pos_unigrams.txt")
pos_bigrams   = get_vocab("vocab/non_static/pos_bigrams/pan/pos_bigrams.txt")
func_words    = get_vocab("vocab/static/function_words.txt")
punc          = get_vocab("vocab/static/punc_marks.txt")
letters       = get_vocab("vocab/static/letters.txt")
common_emojis = get_vocab("vocab/static/common_emojis.txt")
doc_stats     = ["short_words", "large_words", "word_len_avg", "word_len_std", "sent_len_avg", "sent_len_std", "hapaxes"]
deps          = get_vocab("vocab/static/dep_labels.txt")
mixed_bigrams = get_vocab("vocab/non_static/mixed_bigrams/pan/mixed_bigrams.txt")


#TODO: include discourse type as a categorical variable (do tomorrow 02/20)
all_features = pos_unigrams + pos_bigrams + func_words + punc + letters + common_emojis + doc_stats + deps + mixed_bigrams

print(len(all_features))
print(len({feat:[] for feat in all_features}))



def feature_to_count_map(all_features:list[str], feature_vectors:list) -> dict[str,list]:
    
    feats_to_counts = defaultdict(lambda:[])
    for feature in feature_vectors:
        for count_dict in feature.count_map.values():
            for feat_name, count in count_dict.items():
                feats_to_counts[feat_name].append(count)
               
    return feats_to_counts


#counts = feature_to_count_map(all_features, feature_vectors)






407
395


In [19]:
from nltk import bigrams
from more_itertools import collapse
tokens = ["a", "b", "c", "d", "e", "f"]
f = list(bigrams(tokens))
list(collapse(f))



['a', 'b', 'b', 'c', 'c', 'd', 'd', 'e', 'e', 'f']